### Manipulation of PEAKS de novo results of Trocas 8 (low water, November 2019) non-incubation samples LC-MS/MS data using python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >50% ALC

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 675) for another
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
    cd /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/680_Trocas8_June2021_DENOVO_107/

/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/680_Trocas8_June2021_DENOVO_107


In [3]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks680 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/680_Trocas8_June2021_DENOVO_107/680_Trocas8_June2021_DENOVO_107_DN50.csv")

print("# redundant Peaks peptides >50% ALC in combined dataframe:", len(peaks680))

print(peaks680.columns)

# These columns mess things up- get rid of them

del peaks680['Fraction']
del peaks680['Scan']
del peaks680['Source File']
del peaks680['Tag Length']
del peaks680['PTM']
del peaks680['tag (>=0%)']
del peaks680['mode']
del peaks680['local confidence (%)']


columns = ['Peptide', 'ALC', 'length', 'm/z', 'z', 'RT', 'Area',
       'Mass', 'ppm']

peaks680.columns = columns

mean_len = peaks680['length'].mean()
print(mean_len)

# look at the dataframe
peaks680.head()

# redundant Peaks peptides >50% ALC in combined dataframe: 58
Index(['Fraction', 'Scan', 'Source File', 'Peptide', 'Tag Length', 'ALC (%)',
       'length', 'm/z', 'z', 'RT', 'Area', 'Mass', 'ppm', 'PTM',
       'local confidence (%)', 'tag (>=0%)', 'mode'],
      dtype='object')
9.293103448275861


,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm
0,LSSPATLNSR,98,10,523.2849,2,43.99,12700000.0,1044.5564,-1.2
1,LSSPATLDSR,98,10,523.7763,2,47.09,160000.0,1045.5403,-2.1
2,LSSPATLNSR,98,10,523.2863,2,43.48,12700000.0,1044.5564,1.6
3,SPATLNSR,97,8,423.2262,2,22.96,27400.0,844.4402,-2.8
4,LSSPATLNSR,96,10,523.2843,2,44.54,12700000.0,1044.5564,-2.2


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ

We'll then write this manipulated dataframe to a new file.

In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks680['A'] = peaks680['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks680['C'] = peaks680['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks680['D'] = peaks680['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks680['E'] = peaks680['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks680['F'] = peaks680['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks680['G'] = peaks680['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks680['H'] = peaks680['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks680 output, there will be no isoleucines (they're lumped in with leucines)
peaks680['I'] = peaks680['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks680['K'] = peaks680['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks680['L'] = peaks680['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks680['M'] = peaks680['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks680['N'] = peaks680['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks680['P'] = peaks680['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks680['Q'] = peaks680['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks680['R'] = peaks680['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks680['S'] = peaks680['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks680['T'] = peaks680['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks680['V'] = peaks680['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks680['W'] = peaks680['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks680['Y'] = peaks680['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks680['c-carb'] = peaks680['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks680['m-oxid'] = peaks680['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks680['n-deam'] = peaks680['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks680['q-deam'] = peaks680['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaks680['stripped_peptide'] = peaks680['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks680['stripped_length'] = peaks680['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks680['ptm-total'] = peaks680['c-carb'] + peaks680['m-oxid'] + peaks680['n-deam'] + peaks680['q-deam']

# calculate NAAF numerator for each peptide k
peaks680['NAAF_num.'] = peaks680['Area'] / peaks680['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks680.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/680_SMCP_surf_GF75_DN/680_SMCP_surf_GF75_DN50.csv")

# check out the results
peaks680.head()

,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm,A,...,W,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,LSSPATLNSR,98,10,523.2849,2,43.99,12700000.0,1044.5564,-1.2,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,1270000.0
1,LSSPATLDSR,98,10,523.7763,2,47.09,160000.0,1045.5403,-2.1,1,...,0,0,0,0,0,0,LSSPATLDSR,10,0,16000.0
2,LSSPATLNSR,98,10,523.2863,2,43.48,12700000.0,1044.5564,1.6,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,1270000.0
3,SPATLNSR,97,8,423.2262,2,22.96,27400.0,844.4402,-2.8,1,...,0,0,0,0,0,0,SPATLNSR,8,0,3425.0
4,LSSPATLNSR,96,10,523.2843,2,44.54,12700000.0,1044.5564,-2.2,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,1270000.0


In [6]:
# keep only stripped peptide I/L and NAAF
dn_680 = peaks680[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_680.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_680.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/I-L_NAAFs/680_SMCP_surf_GF75_DN50_ILnaafs.csv")

dn_680.head()

,stripped_peptide,Area,NAAF_num.
0,LSSPATLNSR,12700000.0,1270000.0
1,LSSPATLDSR,160000.0,16000.0
2,LSSPATLNSR,12700000.0,1270000.0
3,SPATLNSR,27400.0,3425.0
4,LSSPATLNSR,12700000.0,1270000.0


In [7]:
# made a new dataframe that contains the sums of certain columns 
# in the stripped peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks680['A'].sum(),
        'C': peaks680['C'].sum(),
        'D': peaks680['D'].sum(),
        'E': peaks680['E'].sum(),
        'F': peaks680['F'].sum(),
        'G': peaks680['G'].sum(),
        'H': peaks680['H'].sum(),
        'I': peaks680['I'].sum(),
        'K': peaks680['K'].sum(),
        'L': peaks680['L'].sum(),
        'M': peaks680['M'].sum(),
        'N': peaks680['N'].sum(),
        'P': peaks680['P'].sum(),
        'Q': peaks680['Q'].sum(),
        'R': peaks680['R'].sum(),
        'S': peaks680['S'].sum(),
        'T': peaks680['T'].sum(),
        'V': peaks680['V'].sum(),
        'W': peaks680['W'].sum(),
        'Y': peaks680['Y'].sum(),
        'c-carb': peaks680['c-carb'].sum(),
        'm-oxid': peaks680['m-oxid'].sum(),
        'n-deam': peaks680['n-deam'].sum(),
        'q-deam': peaks680['q-deam'].sum(),
        'Total area': peaks680['Area'].sum(),
        'Total length': peaks680['stripped_length'].sum()
       }

totalpeaks680 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'n-deam', 'q-deam', \
                                            'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks680['% C w/ carb'] = totalpeaks680['c-carb'] / totalpeaks680['C'] 

# calculate percentage of M's that are oxidized
totalpeaks680['% M w/ oxid'] = totalpeaks680['m-oxid'] / totalpeaks680['M'] 

# calculate percentage of N's that are deamidated
totalpeaks680['% N w/ deam'] = totalpeaks680['n-deam'] / totalpeaks680['N'] 

# calculate percentage of N's that are deamidated
totalpeaks680['% Q w/ deam'] = totalpeaks680['q-deam'] / totalpeaks680['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks680['NAAF denom.'] = totalpeaks680['Total area'] / totalpeaks680['Total length']

# write modified dataframe to new txt file
totalpeaks680.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/680_SMCP_surf_GF75_DN/680_SMCP_surf_GF75_DN50_totals.csv")

totalpeaks680.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,39,5,11,14,8,9,40,0,27,75,...,1,2,0,111132403.7,536,1.0,0.027778,0.142857,0.0,207336.574067


In [8]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF50 = 207336.574067

# use NAAF >50% ALC to get NAAF factor
peaks680['NAAF factor'] = (peaks680['NAAF_num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksAAPTM_680 = peaks680[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'n-deam', 'q-deam']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_680['A-NAAF50'] = peaksAAPTM_680['A'] * peaks680['NAAF factor']
peaksAAPTM_680['C-NAAF50'] = peaksAAPTM_680['C'] * peaks680['NAAF factor']
peaksAAPTM_680['D-NAAF50'] = peaksAAPTM_680['D'] * peaks680['NAAF factor']
peaksAAPTM_680['E-NAAF50'] = peaksAAPTM_680['E'] * peaks680['NAAF factor']
peaksAAPTM_680['F-NAAF50'] = peaksAAPTM_680['F'] * peaks680['NAAF factor']
peaksAAPTM_680['G-NAAF50'] = peaksAAPTM_680['G'] * peaks680['NAAF factor']
peaksAAPTM_680['H-NAAF50'] = peaksAAPTM_680['H'] * peaks680['NAAF factor']
peaksAAPTM_680['I-NAAF50'] = peaksAAPTM_680['I'] * peaks680['NAAF factor']
peaksAAPTM_680['K-NAAF50'] = peaksAAPTM_680['K'] * peaks680['NAAF factor']
peaksAAPTM_680['L-NAAF50'] = peaksAAPTM_680['L'] * peaks680['NAAF factor']
peaksAAPTM_680['M-NAAF50'] = peaksAAPTM_680['M'] * peaks680['NAAF factor']
peaksAAPTM_680['N-NAAF50'] = peaksAAPTM_680['N'] * peaks680['NAAF factor']
peaksAAPTM_680['P-NAAF50'] = peaksAAPTM_680['P'] * peaks680['NAAF factor']
peaksAAPTM_680['Q-NAAF50'] = peaksAAPTM_680['Q'] * peaks680['NAAF factor']
peaksAAPTM_680['R-NAAF50'] = peaksAAPTM_680['R'] * peaks680['NAAF factor']
peaksAAPTM_680['S-NAAF50'] = peaksAAPTM_680['S'] * peaks680['NAAF factor']
peaksAAPTM_680['T-NAAF50'] = peaksAAPTM_680['T'] * peaks680['NAAF factor']
peaksAAPTM_680['V-NAAF50'] = peaksAAPTM_680['V'] * peaks680['NAAF factor']
peaksAAPTM_680['W-NAAF50'] = peaksAAPTM_680['W'] * peaks680['NAAF factor']
peaksAAPTM_680['Y-NAAF50'] = peaksAAPTM_680['Y'] * peaks680['NAAF factor']

# multiply the NAAF50 factor by the PTMs normalize its abundance by peak area and peptide length

peaksAAPTM_680['ccarb-NAAF50'] = peaksAAPTM_680['c-carb'] * peaksAAPTM_680['NAAF factor']
peaksAAPTM_680['moxid-NAAF50'] = peaksAAPTM_680['m-oxid'] * peaksAAPTM_680['NAAF factor']
peaksAAPTM_680['ndeam-NAAF50'] = peaksAAPTM_680['n-deam'] * peaksAAPTM_680['NAAF factor']
peaksAAPTM_680['qdeam-NAAF50'] = peaksAAPTM_680['q-deam'] * peaksAAPTM_680['NAAF factor']


# write the dataframe to a new csv
peaksAAPTM_680.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/680_SMCP_surf_GF75_DN/680_SMCP_surf_GF75_DN50_naaf.csv")

peaksAAPTM_680.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,K,...,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50
0,LSSPATLNSR,6.125306,1,0,0,0,0,0,0,0,...,6.125306,18.375919,6.125306,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,LSSPATLDSR,0.077169,1,0,1,0,0,0,0,0,...,0.077169,0.231508,0.077169,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,LSSPATLNSR,6.125306,1,0,0,0,0,0,0,0,...,6.125306,18.375919,6.125306,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SPATLNSR,0.016519,1,0,0,0,0,0,0,0,...,0.016519,0.033038,0.016519,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,LSSPATLNSR,6.125306,1,0,0,0,0,0,0,0,...,6.125306,18.375919,6.125306,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMs

index = ['sample total']

data = {'NAAF': peaksAAPTM_680['NAAF factor'].sum(),
        'A': peaksAAPTM_680['A-NAAF50'].sum(),
        'C': peaksAAPTM_680['C-NAAF50'].sum(),
        'D': peaksAAPTM_680['D-NAAF50'].sum(),
        'E': peaksAAPTM_680['E-NAAF50'].sum(),
        'F': peaksAAPTM_680['F-NAAF50'].sum(),
        'G': peaksAAPTM_680['G-NAAF50'].sum(),
        'H': peaksAAPTM_680['H-NAAF50'].sum(),
        'I': peaksAAPTM_680['I-NAAF50'].sum(),
        'K': peaksAAPTM_680['K-NAAF50'].sum(),
        'L': peaksAAPTM_680['L-NAAF50'].sum(),
        'M': peaksAAPTM_680['M-NAAF50'].sum(),
        'N': peaksAAPTM_680['N-NAAF50'].sum(),
        'P': peaksAAPTM_680['P-NAAF50'].sum(),
        'Q': peaksAAPTM_680['Q-NAAF50'].sum(),
        'R': peaksAAPTM_680['R-NAAF50'].sum(),
        'S': peaksAAPTM_680['S-NAAF50'].sum(),
        'T': peaksAAPTM_680['T-NAAF50'].sum(),
        'V': peaksAAPTM_680['V-NAAF50'].sum(),
        'W': peaksAAPTM_680['W-NAAF50'].sum(),
        'Y': peaksAAPTM_680['Y-NAAF50'].sum(),
        'c-carb': peaksAAPTM_680['ccarb-NAAF50'].sum(),
        'm-oxid': peaksAAPTM_680['moxid-NAAF50'].sum(),
        'n-deam': peaksAAPTM_680['ndeam-NAAF50'].sum(),
        'q-deam': peaksAAPTM_680['qdeam-NAAF50'].sum(),
       }

totalpeaks50_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks50_NAAF['% C w/ carb.'] = totalpeaks50_NAAF['c-carb'] / totalpeaks50_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks50_NAAF['% M w/ oxid'] = totalpeaks50_NAAF['m-oxid'] / totalpeaks50_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% N w/ deam'] = totalpeaks50_NAAF['n-deam'] / totalpeaks50_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% Q w/ deam'] = totalpeaks50_NAAF['q-deam'] / totalpeaks50_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks50_NAAF['NAAF check'] = totalpeaks50_NAAF['NAAF'] / 207336.574067

# write modified dataframe to new txt file, same name + totals
totalpeaks50_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/680_SMCP_surf_GF75_DN/680_SMCP_surf_GF75_DN50_naaf_totals.csv")

totalpeaks50_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,60.731583,45.60985,0.069018,0.519664,0.114507,0.208971,2.924819,3.551062,0.0,2.663233,...,1.814462,0.069018,0.021173,0.369004,0.0,1.0,0.00786,0.014736,0.0,0.000293


## Export stripped peptides >50% ALC

In [10]:
###### keep only stripped peptide column 
pep50 = peaks680[["stripped_peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/680_SMCP_surf_GF75_DN/680_SMCP_surf_GF75_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/680_SMCP_surf_GF75_DN/680_SMCP_surf_GF75_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/680_SMCP_surf_GF75_DN/680_SMCP_surf_GF75_DN50_stripped_peptides.fas

# removing redundancy
peaks50dedup = pd.DataFrame.drop_duplicates(pep50)

print("# redundant stripped Peaks peptides >50% ALC", len(pep50))
print("# nonredundant stripped Peaks peptides >50% ALC", len(peaks50dedup))
print("average peptide length Peaks peptides >50% ALC", peaks680['stripped_length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >50% ALC
pep50m = peaks680[["Peptide"]]

# deduplicate
pep50mdedup = pd.DataFrame.drop_duplicates(pep50m)

print("# redundant Peaks peptides >50% ALC", len(pep50m))
print("# nonredundant Peaks peptides", len(pep50mdedup))

# check
pep50.head()

# redundant stripped Peaks peptides >50% ALC 58
# nonredundant stripped Peaks peptides >50% ALC 41
average peptide length Peaks peptides >50% ALC 9.241379310344827
# redundant Peaks peptides >50% ALC 58
# nonredundant Peaks peptides 41


,stripped_peptide
0,LSSPATLNSR
1,LSSPATLDSR
2,LSSPATLNSR
3,SPATLNSR
4,LSSPATLNSR
